In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import KFold
import tensorflow as tf

Read in images and do some preprocessing. X is images, Y is the animal designation. 
* 0 - Bighorn Sheep
* 1 - Bobcat
* 2 - Coyote
* 3 - Gray Fox
* 4 - Javelina
* 5 - Mule Deer
* 6 - Raptor
* 7 - White-tailed Deer

In [2]:
x = np.zeros((2112,256,256,3))
y = np.zeros((2112, 1))
count = 0
for root, dirs, files in os.walk(".\cs510chips"):
    for filename in files:
        animal = 0
        if "Bighorn_Sheep" in filename:
            animal = 0
        if "Bobcat" in filename:
            animal = 1
        if "Coyote" in filename:
            animal = 2
        if "Gray_Fox" in filename:
            animal = 3
        if "Javelina" in filename:
            animal = 4
        if "Mule_Deer" in filename:
            animal = 5
        if "Raptor" in filename:
            animal = 6
        if "White_tailed_Deer" in filename:
            animal = 7

        fn = '.\cs510chips\{}'.format(filename)
        im = cv2.imread(fn)
        if(type(im) != type(None)):
            if(im.shape == (256,256,3)):
                x[count] = im
                y[count] = animal
                count = count + 1

Checking the data sets

In [3]:
x.shape, y.shape

((2112, 256, 256, 3), (2112, 1))

In [4]:
cv2.imshow("test", np.array(x[0], dtype=np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

3-fold cross validation split for testing and training.

In [5]:
kf = KFold(n_splits=3)
kf.get_n_splits(x)
for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

Defining the model as an instance of keras resnet.

In [6]:
model = tf.keras.applications.resnet.ResNet101(include_top=False, weights='imagenet', input_shape=(256,256,3), pooling='avg')

Calculating log-odds scores for each class.

In [7]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[0.07387878, 0.570175  , 1.6539543 , ..., 0.04927567, 0.12244673,
        0.1352936 ]], dtype=float32)

Converting log-odds scores to probabilities for each class.

In [8]:
tf.nn.softmax(predictions).numpy()

array([[3.8720460e-05, 6.3603213e-05, 1.8800028e-04, ..., 3.7779413e-05,
        4.0647439e-05, 4.1173007e-05]], dtype=float32)

Returns a scalar loss for each example, a negative log probability of the true class. 

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
loss_fn(y_train[:1], predictions).numpy()

9.662847

In [11]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Fitting the model to the actual data.

In [12]:
model.fit(x_train, y_train, epochs=5)

Train on 1408 samples
Epoch 1/10
  64/1408 [>.............................] - ETA: 4:57:59 - loss: 7.9436 - accuracy: 0.0000e+00

KeyboardInterrupt: 

Seeing how well the model performs on the test data.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)